This is a pretty straightforward text generation task, using a fine-tuned version of the GPT-2 language model to quickly come up with some amusing texts that look relatively close to actual bug reports, albeit a little skewed.

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import gpt_2_simple as gpt2
import os
import tensorflow as tf

C:\Users\Rich\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Rich\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Rich\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Rich\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

Do yourself a favour and run this on a GPU. I haven't timed it CPU-only, since I think I'll probably need my computer in the next few days.

In [2]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

['/device:CPU:0', '/device:GPU:0']


The next few cells scrape the text of various bug reports from the Dwarf Fortress tracker... 

In [2]:
def get_bugs_page(pageno):
    r = requests.get(f'https://www.bay12games.com/dwarves/mantisbt/view_all_bug_page.php?page_number={pageno}')
    if r.status_code == 200:
        return r.text
    return False

In [3]:
def extract_bug_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    tds = soup.find_all('td', class_='left')
    return [td.text.strip() for td in tds[:-1] if len(td.text) > 5] 

...and fire them into a text file, separated by newlines, for use in training our model.

In [6]:
for i in range(1, 215):
    text = get_bugs_page(i)
    bugs = extract_bug_text(text)
    with open('bugs.txt', 'a', encoding='utf-8') as outfile:
        for bug in bugs:
            outfile.write(f'{bug}\n')

I'm using the 3.55 million parameter gpt-2 model here, which clocks in at about 1.5GB before fine-tuning.

In [4]:
model_name = "355M"
if not os.path.isdir(os.path.join("models", model_name)):
    gpt2.download_gpt2(model_name=model_name)

I've set the fine-tuning to start from scratch every time, without restoring from a previous model run. If you want to restore, just remove the restore_from param. It will also give a sample output every 200 training steps. I'm only training for 1,000 steps here, but you could probably get better results with more training - although there's always the risk of creating spurious connections from training with a restricted task dataset.

In [5]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess, 'bugs.txt', model_name=model_name, steps=1000, restore_from='fresh', sample_every=200)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models\355M\model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models\355M\model.ckpt


  0%|                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.28s/it]


dataset has 134144 tokens
Training...
[1 | 11.71] loss=4.57 avg=4.57
[2 | 12.73] loss=4.70 avg=4.63
[3 | 13.75] loss=4.63 avg=4.63
[4 | 14.76] loss=4.61 avg=4.62
[5 | 15.78] loss=4.33 avg=4.56
[6 | 16.81] loss=4.70 avg=4.59
[7 | 17.84] loss=4.28 avg=4.54
[8 | 18.89] loss=4.55 avg=4.54
[9 | 19.92] loss=4.74 avg=4.57
[10 | 20.94] loss=4.72 avg=4.58
[11 | 21.95] loss=4.56 avg=4.58
[12 | 22.97] loss=4.64 avg=4.58
[13 | 23.99] loss=4.19 avg=4.55
[14 | 25.01] loss=4.43 avg=4.54
[15 | 26.03] loss=4.31 avg=4.53
[16 | 27.05] loss=4.44 avg=4.52
[17 | 28.08] loss=4.43 avg=4.51
[18 | 29.09] loss=4.69 avg=4.53
[19 | 30.11] loss=4.52 avg=4.52
[20 | 31.13] loss=4.57 avg=4.53
[21 | 32.15] loss=4.38 avg=4.52
[22 | 33.17] loss=4.42 avg=4.51
[23 | 34.19] loss=4.44 avg=4.51
[24 | 35.21] loss=4.55 avg=4.51
[25 | 36.23] loss=4.11 avg=4.49
[26 | 37.25] loss=4.44 avg=4.49
[27 | 38.28] loss=4.26 avg=4.48
[28 | 39.30] loss=4.41 avg=4.48
[29 | 40.37] loss=4.03 avg=4.46
[30 | 41.39] loss=4.33 avg=4.46
[31 | 42.41

[201 | 242.32] loss=4.25 avg=3.67
[202 | 243.37] loss=2.85 avg=3.66
[203 | 244.39] loss=2.34 avg=3.64
[204 | 245.41] loss=2.23 avg=3.62
[205 | 246.46] loss=4.44 avg=3.63
[206 | 247.51] loss=3.90 avg=3.64
[207 | 248.56] loss=3.69 avg=3.64
[208 | 249.59] loss=2.04 avg=3.62
[209 | 250.62] loss=4.07 avg=3.62
[210 | 251.65] loss=3.30 avg=3.62
[211 | 252.67] loss=1.98 avg=3.60
[212 | 253.71] loss=3.09 avg=3.60
[213 | 254.75] loss=3.21 avg=3.59
[214 | 255.79] loss=3.73 avg=3.59
[215 | 256.81] loss=2.76 avg=3.58
[216 | 257.85] loss=1.53 avg=3.56
[217 | 258.90] loss=3.88 avg=3.56
[218 | 259.96] loss=3.36 avg=3.56
[219 | 261.01] loss=3.55 avg=3.56
[220 | 262.08] loss=1.74 avg=3.54
[221 | 263.12] loss=4.14 avg=3.55
[222 | 264.18] loss=3.59 avg=3.55
[223 | 265.24] loss=3.83 avg=3.55
[224 | 266.28] loss=4.26 avg=3.56
[225 | 267.33] loss=2.21 avg=3.54
[226 | 268.39] loss=3.31 avg=3.54
[227 | 269.42] loss=2.93 avg=3.54
[228 | 270.44] loss=4.41 avg=3.55
[229 | 271.46] loss=3.11 avg=3.54
[230 | 272.59]

[401 | 474.68] loss=2.53 avg=2.82
[402 | 475.77] loss=2.40 avg=2.81
[403 | 476.79] loss=1.96 avg=2.80
[404 | 477.82] loss=2.27 avg=2.80
[405 | 478.89] loss=2.64 avg=2.80
[406 | 479.95] loss=1.54 avg=2.78
[407 | 481.03] loss=1.63 avg=2.77
[408 | 482.07] loss=2.21 avg=2.77
[409 | 483.10] loss=1.71 avg=2.76
[410 | 484.13] loss=1.16 avg=2.74
[411 | 485.15] loss=1.80 avg=2.73
[412 | 486.18] loss=1.30 avg=2.72
[413 | 487.20] loss=1.98 avg=2.71
[414 | 488.23] loss=0.92 avg=2.69
[415 | 489.25] loss=1.89 avg=2.68
[416 | 490.28] loss=2.54 avg=2.68
[417 | 491.32] loss=1.85 avg=2.67
[418 | 492.38] loss=2.04 avg=2.67
[419 | 493.43] loss=1.74 avg=2.66
[420 | 494.49] loss=1.61 avg=2.65
[421 | 495.53] loss=1.07 avg=2.63
[422 | 496.55] loss=2.42 avg=2.63
[423 | 497.58] loss=1.25 avg=2.61
[424 | 498.61] loss=2.42 avg=2.61
[425 | 499.67] loss=2.00 avg=2.61
[426 | 500.70] loss=1.90 avg=2.60
[427 | 501.78] loss=2.87 avg=2.60
[428 | 502.86] loss=2.64 avg=2.60
[429 | 503.93] loss=0.87 avg=2.58
[430 | 504.95]

[601 | 705.85] loss=0.51 avg=1.71
[602 | 706.87] loss=0.60 avg=1.70
[603 | 707.90] loss=1.42 avg=1.69
[604 | 708.92] loss=1.48 avg=1.69
[605 | 709.95] loss=1.10 avg=1.69
[606 | 710.98] loss=0.95 avg=1.68
[607 | 712.00] loss=1.04 avg=1.67
[608 | 713.03] loss=0.69 avg=1.66
[609 | 714.10] loss=0.89 avg=1.65
[610 | 715.16] loss=1.88 avg=1.66
[611 | 716.18] loss=0.53 avg=1.65
[612 | 717.21] loss=0.66 avg=1.64
[613 | 718.25] loss=0.55 avg=1.62
[614 | 719.30] loss=0.55 avg=1.61
[615 | 720.34] loss=0.76 avg=1.61
[616 | 721.39] loss=1.98 avg=1.61
[617 | 722.43] loss=1.97 avg=1.61
[618 | 723.50] loss=1.61 avg=1.61
[619 | 724.55] loss=0.39 avg=1.60
[620 | 725.59] loss=4.62 avg=1.63
[621 | 726.63] loss=0.64 avg=1.62
[622 | 727.67] loss=1.88 avg=1.62
[623 | 728.71] loss=1.42 avg=1.62
[624 | 729.76] loss=4.16 avg=1.65
[625 | 730.79] loss=2.41 avg=1.65
[626 | 731.83] loss=0.50 avg=1.64
[627 | 732.88] loss=1.92 avg=1.65
[628 | 733.92] loss=0.54 avg=1.63
[629 | 734.96] loss=1.26 avg=1.63
[630 | 736.04]

[801 | 938.98] loss=1.18 avg=1.10
[802 | 940.02] loss=1.79 avg=1.11
[803 | 941.07] loss=1.23 avg=1.11
[804 | 942.10] loss=0.35 avg=1.10
[805 | 943.14] loss=0.97 avg=1.10
[806 | 944.19] loss=0.55 avg=1.09
[807 | 945.22] loss=1.21 avg=1.09
[808 | 946.25] loss=0.65 avg=1.09
[809 | 947.27] loss=1.22 avg=1.09
[810 | 948.32] loss=1.21 avg=1.09
[811 | 949.37] loss=0.43 avg=1.08
[812 | 950.41] loss=3.53 avg=1.11
[813 | 951.46] loss=0.93 avg=1.11
[814 | 952.49] loss=0.67 avg=1.10
[815 | 953.54] loss=0.55 avg=1.10
[816 | 954.60] loss=0.45 avg=1.09
[817 | 955.64] loss=0.53 avg=1.09
[818 | 956.69] loss=0.30 avg=1.08
[819 | 957.71] loss=0.87 avg=1.08
[820 | 958.76] loss=0.40 avg=1.07
[821 | 959.79] loss=0.19 avg=1.06
[822 | 960.82] loss=1.38 avg=1.06
[823 | 961.85] loss=0.23 avg=1.06
[824 | 962.91] loss=2.90 avg=1.07
[825 | 963.95] loss=0.64 avg=1.07
[826 | 964.98] loss=0.74 avg=1.07
[827 | 966.01] loss=1.95 avg=1.07
[828 | 967.07] loss=0.41 avg=1.07
[829 | 968.13] loss=0.51 avg=1.06
[830 | 969.17]

We can now generate with our fine-tuned language model. We'll look for 1,000 samples of up to 100 tokens, and gpt-2 allows us to run batches in parallel. The generation is non-deterministic (check here for an explanation of why that is https://huggingface.co/blog/how-to-generate), so we won't get the same results at each run time. I've pumped up the temperature (the likelihood of low_probability words being included in our sequences) to get slightly crazier results, which I find amusing.

In [31]:
texts = gpt2.generate(sess, length=100, nsamples=1000, batch_size=10, temperature=1.0, top_p=0.9, return_as_list=True)

There's a couple of problems we need to fix before saving our results. The model will generate a chunk of text up to the desired token length, but we want a nice separated list of potential messages. So, we split the sequence on newline characters, throwing away any blank results. Then, we get rid of the last result. Why? Because once the token limit is reached, generation will stop in the middle of a phrase, which we don't want to include.

In [32]:
with open('gen_texts.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for text in texts:
        split_texts = [i for i in text.split('\n') if len(i) > 0][:-1]
        for msg in split_texts:
            writer.writerow([msg])